# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

**1. Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`.**

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**2. Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**3. Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**4. Выведем количество пропущенных значений для каждого столбца, используя комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**5. Заполним пропуски в столбце `total_income` медианным значением по каждому типу из столбца `income_type`.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**6. Заменим все отрицательные значения в столбце `days_employed` положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**7. Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Оставим их как есть, так как этот столбец не понадобится нам для исследования.

**8. Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**9. В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**10. Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**11. Заполним пропуски в столбце `days_employed` медианными значениями для каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**12. Ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**13. Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

<ipython-input-13-990472087ee4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['total_income'] = data['total_income'].astype(int)


### Обработка дубликатов

**14. Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.**

In [14]:
data['education'] = data['education'].str.lower()

<ipython-input-14-d4b849f854d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['education'] = data['education'].str.lower()


**15. Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**16. На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Используем для этого собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

<ipython-input-18-0bc713f3049f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['total_income_category'] = data['total_income'].apply(categorize_income)


**17. Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**18. Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

<ipython-input-21-ecf9d79505dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['purpose_category'] = data['purpose'].apply(categorize_purpose)


### 3. Исследовательский анализ данных

#### 3.1 Определим, есть ли зависимость между количеством детей и возвратом кредита в срок

In [22]:
# Сгруппируем в таблицу и вычислим общее количество должников для определенного числа детей:
debtor_quantity = data.groupby(['children']).agg({'debt':'sum'})

# Теперь сгруппируем в таблицу и вычислим общее количество заемщиков для определенного числа детей:
borrower_quantity = data.groupby(['children']).agg({'family_status_id':'count'})

# И определим процент должников в каждой группе:
debtor_percentage = (debtor_quantity['debt'] / borrower_quantity['family_status_id']) * 100

# Отсортируем по возрастанию:
debtor_percentage.sort_values()

children
5    0.000000
0    7.543822
3    8.181818
1    9.234609
2    9.454191
4    9.756098
dtype: float64

In [23]:
# Теперь сформируем по полученным данным сводную таблицу.

# Добавим в таблицу debtor_quantity столбец с общим количеством заемщиков:
debtor_quantity['borrower_quantity'] = data.groupby(['children']).agg({'family_status_id':'count'})

# Далее добавим в нее же столбец со средним процентом просрочки:
debtor_quantity['debtor_percentage'] = debtor_quantity['debt'] / debtor_quantity['borrower_quantity'] * 100

# И выведем получившуюся сводную таблицу на экран:
debtor_quantity

,debt,borrower_quantity,debtor_percentage
children,,,
0,1063,14091,7.543822
1,444,4808,9.234609
2,194,2052,9.454191
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000


**Вывод:**
Для формулирования вывода стоит ограничиться тремя верхними строками сводной таблицы. Нижние три, в силу их малочисленности, трудно считать репрезентативными.

По полученным данным видно, что средний процент должников немного ниже среди заемщиков без детей. В категории заемщиков с одним ребенком число должников возрастает на 1,7%. И еще на 0,2% для заемщиков с двумя детьми. Разница в 0,2% слишком мала, чтобы однозначно вывести какую-то зависимость. Таким образом, стоит ограничиться лишь выводом, что процент невозврата среди заемщиков без детей, в целом, ниже, чем среди заемщиков с детьми.

#### 3.2 Определим, есть ли зависимость между семейным положением и возвратом кредита в срок

In [24]:
# Сгруппируем в таблицу и вычислим общее количество должников для определенного семейного положения:
debtor_quantity = data.groupby(['family_status']).agg({'debt':'sum'})

# Теперь сгруппируем в таблицу и вычислим общее количество заемщиков для определенного семейного положения:
borrower_quantity = data.groupby(['family_status']).agg({'family_status_id':'count'})

# И определим процент должников в каждой группе:
debtor_percentage = (debtor_quantity['debt'] / borrower_quantity['family_status_id']) * 100

# Отсортируем по возрастанию:
debtor_percentage.sort_values()

family_status
вдовец / вдова           6.624606
в разводе                7.064760
женат / замужем          7.560558
гражданский брак         9.313014
Не женат / не замужем    9.763948
dtype: float64

In [25]:
# Теперь сформируем по полученным данным сводную таблицу.

# Добавим в таблицу debtor_quantity столбец с общим количеством заемщиков:
debtor_quantity['borrower_quantity'] = data.groupby(['family_status']).agg({'family_status_id':'count'})

# Далее добавим в нее же столбец со средним процентом просрочки:
debtor_quantity['debtor_percentage'] = debtor_quantity['debt'] / debtor_quantity['borrower_quantity'] * 100

# И выведем получившуюся сводную таблицу на экран:
debtor_quantity.sort_values('debtor_percentage')

,debt,borrower_quantity,debtor_percentage
family_status,,,
вдовец / вдова,63,951,6.624606
в разводе,84,1189,7.064760
женат / замужем,927,12261,7.560558
гражданский брак,385,4134,9.313014
Не женат / не замужем,273,2796,9.763948


**Вывод:**
Исходя из полученных данных можно заметить, что процент должников немного ниже среди заемщиков, состоящих или ранее состоявщих в официальном браке. Должников среди заемщиков, живущих в гражданском браке или никогда не бывших женатыми/замужем, на два-три процента больше.

#### 3.3 Определим, есть ли зависимость между уровнем дохода и возвратом кредита в срок

In [26]:
# Сгруппируем в таблицу и вычислим общее количество должников для определенной категории дохода:
debtor_quantity = data.groupby(['total_income_category']).agg({'debt':'sum'})

# Теперь сгруппируем в таблицу и вычислим общее количество заемщиков для определенной категории дохода:
borrower_quantity = data.groupby(['total_income_category']).agg({'family_status_id':'count'})

# И определим процент должников в каждой категории:
debtor_percentage = (debtor_quantity['debt'] / borrower_quantity['family_status_id']) * 100

# Отсортируем по возрастанию:
debtor_percentage.sort_values()

total_income_category
D    6.017192
B    7.060231
A    8.000000
C    8.498210
E    9.090909
dtype: float64

In [27]:
# Теперь сформируем по полученным данным сводную таблицу.

# Добавим в таблицу debtor_quantity столбец с общим количеством заемщиков:
debtor_quantity['borrower_quantity'] = data.groupby(['total_income_category']).agg({'family_status_id':'count'})

# Далее добавим в нее же столбец со средним процентом просрочки:
debtor_quantity['debtor_percentage'] = debtor_quantity['debt'] / debtor_quantity['borrower_quantity'] * 100

# И выведем получившуюся сводную таблицу на экран:
debtor_quantity.sort_values('borrower_quantity', ascending=False)

,debt,borrower_quantity,debtor_percentage
total_income_category,,,
C,1353,15921,8.498210
B,354,5014,7.060231
D,21,349,6.017192
A,2,25,8.000000
E,2,22,9.090909


**Вывод:**
Для формулирования вывода стоит ограничиться двумя верхними строками сводной таблицы. Нижние три, в силу их малочисленности, трудно считать репрезентативными.

Полученные данные свидетельствуют о том, что число должников в категории с доходом от 50-ти до 200-т тысяч рублей в среднем на 1,5% выше, чем в категории с доходом от 200-т тысяч до миллиона рублей.

Таким образом, можно предположить, что зависимость обратнопропорциональна: чем выше доход, тем меньше процент просрочек.

#### 3.4 Определим, как разные цели кредита влияют на его возврат в срок

In [28]:
# Сгруппируем в таблицу и вычислим общее количество должников для определенной категории целей кредита:
debtor_quantity = data.groupby(['purpose_category']).agg({'debt':'sum'})

# Теперь сгруппируем в таблицу и вычислим общее количество заемщиков для определенной категории целей кредита:
borrower_quantity = data.groupby(['purpose_category']).agg({'family_status_id':'count'})

# И определим процент должников в каждой категории:
debtor_percentage = (debtor_quantity['debt'] / borrower_quantity['family_status_id']) * 100

# Отсортируем по возрастанию:
debtor_percentage.sort_values()

purpose_category
операции с недвижимостью    7.255139
проведение свадьбы          7.911803
получение образования       9.252758
операции с автомобилем      9.347978
dtype: float64

In [29]:
# Теперь сформируем по полученным данным сводную таблицу.

# Добавим в таблицу debtor_quantity столбец с общим количеством заемщиков:
debtor_quantity['borrower_quantity'] = data.groupby(['purpose_category']).agg({'family_status_id':'count'})

# Далее добавим в нее же столбец со средним процентом просрочки:
debtor_quantity['debtor_percentage'] = debtor_quantity['debt'] / debtor_quantity['borrower_quantity'] * 100

# И выведем получившуюся сводную таблицу на экран:
debtor_quantity.sort_values('debtor_percentage')

,debt,borrower_quantity,debtor_percentage
purpose_category,,,
операции с недвижимостью,780,10751,7.255139
проведение свадьбы,183,2313,7.911803
получение образования,369,3988,9.252758
операции с автомобилем,400,4279,9.347978


**Вывод:**
Наименьшее число должников среди заемщиков, взявших кредит для операций с недвижимостью. Чуть большее - на проведение свадьбы. На два процента больше должников среди тех, кто брал кредит с целью получения образования. И, наконец, самое высокое число невозвратов кредита в срок в категории "операции с автомобилем".

#### 3.5 Возможные причины появления пропусков в исходных данных:

Пропуски в столбцах days_employed и total_income могли появиться из-за нежелания заемщиков корректно указывать свой стаж или реальный уровень дохода. Также, для категорий безработных, пенсионеров и студентов могли возникнуть затруднения с корректным определением этих показателей.

#### 3.6 Обоснование выбора медианных значений для заполнения пропусков:

Чрезмерно большие или маленькие "крайние" значения количественных переменных могут сильно исказить данные при заполнении пропусков средними значениями. Поэтому, в данном случае, лучше заполнять пропуски значением медианным.

### 4. Общий вывод

В данном проекте был использован датасет с информацией о денежных заемщиках: количество у них детей, период занятости, семейный статус, уровень дохода, наличие просрочки и т.д.

На этапе предобработки были обнаружены и заменены медианными значениями пропуски в столбцах days_employed и total_income. Также были устранены аномалии в виде чрезмерно большого значения в столбце children и отрицательных значений в столбцах days_employed и children. В столбце total_income тип данных был изменен с вещественного на целочисленный. А в столбце education потребовалось избавиться от дубликатов. Для удобства дальнейшего исследования была проведена категоризация столбцов total_income и purpose.

Помимо отсутствия указанных выше пропусков и аномалий, на результатах исследования могло бы положительно сказаться более равномерное распределение данных по категориям "дети" и "уровень дохода". В текущем виде датасета, рядом категорий, в виду малочисленности их значений, пришлось пренебречь.

В результате исследования были получены следующие выводы:
1) Процент просрочки среди заемщиков без детей, в целом, на 1,7-2% ниже, чем среди заемщиков с детьми.

2) Процент должников ниже среди заемщиков, состоящих или ранее состоявщих в официальном браке. Должников среди заемщиков, живущих в гражданском браке или никогда не бывших женатыми/замужем, на 2-3% больше.

3) Число должников в категории с доходом от 50-ти до 200-т тысяч рублей в среднем на 1,5% выше, чем в категории с доходом от 200-т тысяч до миллиона рублей. Следовательно, можно предположить, что зависимость обратнопропорциональна: чем выше доход, тем меньше процент просрочек.

4) Наименьшее число должников среди заемщиков, взявших кредит для операций с недвижимостью. Чуть большее - на проведение свадьбы. На 2% больше должников среди тех, кто брал кредит с целью получения образования. И, наконец, самое высокое число невозвратов кредита в срок в категории "операции с автомобилем".

Таким образом, можно сделать вывод, что наиболее надежными заемщиками являются лица с высоким уровнем дохода, на текущий момент не имеющие детей, но состоящие или ранее состоявшие в браке. А также те, кто целью взятия кредита указывает "операции с недвижимостью". Менее надежными, с небольшим отличием в среднем в два процента, можно считать тех граждан, кто ранее не состоял в браке и указывает целью кредита операции с автомобилем и получение образования.

В целом, проведенная выборка по различным категориям дает отличия по количеству должников всего в 1-3%, но, при необходимости, рекомендации, приведенные выше, могут быть использованы для принятия решения о выдаче кредита.